# **1 - Template Configurations**
## Configuations Template
*By Author's Name*

In this notebook we can see visually the effects of changing the values in the config has on the images and how the model will see them.

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import os
import sys
import numpy as np
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
import matplotlib.pyplot as plt

with open("settings.json") as s:
    settings = json.load(s)
    
ROOT_DIR = settings["META"]["ROOT_DIR"]
sys.path.append(ROOT_DIR)

# Configurations

In [ ]:
from roadlearning.configurations import json_loader
config = json_loader(settings["CONFIGS"]["BASE"]) # change "" to config name.
print(config)

# Base Dataset

In [ ]:
from roadlearning.dataset import Dataset

In [ ]:
from roadlearning.augmentations import *
import imgaug.augmenters as iaa
my_augmentation_handler = AugmentationHandler()

class MyAugment(SimpleAugment):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
#        self.add_augmentation("rotation-5", "Rotaion by -5 degrees", self.rotate_by(-5))
#        self.add_augmentation("rotation5", "Rotaion by 5 degrees", self.rotate_by(5))
        
        self.add_augmentation("flip", "hoizonal flip", self.flip_by(1))
        
        self.add_augmentation("hue0", "hue 0, (0, 30)", (self.add_to_hue, lambda x: x))
        self.add_augmentation("hue1", "hue 1, (0, 30)", (self.add_to_saturation, lambda x: x))
#         self.add_augmentation("hue2", "hue 2, (0, 30)", (lambda x: iaa.AddToValue((0, 30))(image = x), lambda x: x))
    @staticmethod
    def add_to_hue(image):
        if len(image.shape) < 3:
            return image
        aug = iaa.AddToHue((0, 30))
        return aug(image=image)
    
    @staticmethod
    def add_to_saturation(image):
        if len(image.shape) < 3:
            return image
        aug = iaa.AddToSaturation((0, 30))
        return aug(image=image)
    
    
    def load_augments(self, image):
        hues = ["original", "hue0", "hue1"]
        flips = ["original", "flip"]
        return [self.aug_combo("original", i) for i in ["original", "hue0", "hue1", ]] 
        
my_augmentation_handler.add_augment("MyAugment", MyAugment) 

In [ ]:
dataset = Dataset(source=config.NAME)
dataset.load_data(**settings["DATASET_LOADS"]["BASE"]) # Change Here

In [ ]:
print(dataset)

### Visualing base dataset information

In [ ]:
from roadlearning.plotting.visualise import heatmap, display_dataset
%matplotlib inline

Sample Images

In [ ]:
sample = 4
size = 14
display_dataset(dataset, sample, size)

Images heatmap

In [ ]:
heatmap(dataset)

### Random Image Id

In [ ]:
image_id = np.random.choice(dataset.image_ids)

### Bounding Boxes

In [ ]:
# Load random image and mask.
image = dataset.load_image(image_id)
plt.imshow(image)
mask, class_ids = dataset.load_mask(image_id)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, dataset.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

### Mini Masks

In [ ]:
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=True)

print("image_id ", image_id, dataset.image_reference(image_id))

log("image", image)
log("mask", mask)
log("image_meta", image_meta)
log("class_ids", class_ids)
log("bbox", bbox)

visualize.display_images([image] + [mask[:,:,i] for i in range(min(mask.shape[-1], 7))], cols=2)

## Augmented Dataset

In [ ]:
dataset = Dataset(source=config.NAME)
dataset.load_data(**settings["DATASET_LOADS"][""]) # Change Here

In [ ]:
print(dataset)

### Visualing Augmented Dataset Information

In [ ]:
from roadlearning.plotting.visualise import heatmap, display_dataset
%matplotlib inline

Sample Images

In [ ]:
sample = 4
size = 14
display_dataset(dataset, sample, size)

Images heatmap

In [ ]:
heatmap(dataset)

### Random Image Id

In [ ]:
image_id = np.random.choice(dataset.image_ids)

### Bounding Boxes

In [ ]:
# Load random image and mask.
image = dataset.load_image(image_id)
mask, class_ids = dataset.load_mask(image_id)
# Compute Bounding box
bbox = utils.extract_bboxes(mask)

# Display image and additional stats
print("image_id ", image_id, dataset.image_reference(image_id))
log("image", image)
log("mask", mask)
log("class_ids", class_ids)
log("bbox", bbox)
# Display image and instances
visualize.display_instances(image, bbox, mask, class_ids, dataset.class_names)

### Mini Masks

In [ ]:
image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=True)

print("image_id ", image_id, dataset.image_reference(image_id))

log("image", image)
log("mask", mask)
log("image_meta", image_meta)
log("class_ids", class_ids)
log("bbox", bbox)

visualize.display_images([image] + [mask[:,:,i] for i in range(min(mask.shape[-1], 7))], cols=2)